In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
features <- read_csv("/kaggle/input/ncaa-2024/features_mens.csv")#This data is created by the following code.(https://www.kaggle.com/code/flat831/create-feature-mens)
result <- read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyCompactResults.csv")

Rows: 5670 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): TeamName
dbl (11): TeamID, FirstD1Season, LastD1Season, Rating, num, Season, count, w...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2451 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): WLoc
dbl (7): Season, DayNum, WTeamID, WScore, LTeamID, LScore, NumOT

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
tmp <- result %>% 
  rename(
    TeamID_A = WTeamID, TeamID_B = LTeamID, Score_A = WScore, Score_B = LScore
    )

tmp2 <- result %>% 
  rename(
    TeamID_A = LTeamID, TeamID_B = WTeamID, Score_A = LScore, Score_B = WScore
  )

features_A <- features %>% 
  select(Season, TeamID, Seed, Rating, win_rate, gap_avg) %>% 
  filter(is.na(Seed) == 0) %>% 
  rename(TeamID_A = TeamID, Seed_A = Seed, Rating_A = Rating, win_rate_A = win_rate, gap_avg_A = gap_avg)

features_B <- features %>% 
  select(Season, TeamID, Seed, Rating, win_rate, gap_avg) %>% 
  filter(is.na(Seed) == 0) %>% 
  rename(TeamID_B = TeamID, Seed_B = Seed, Rating_B = Rating, win_rate_B = win_rate, gap_avg_B = gap_avg)

result_merged <- bind_rows(tmp, tmp2) %>% 
  filter(Season >= 2010) %>% 
  left_join(features_A, by = c("Season", "TeamID_A")) %>% 
  left_join(features_B, by = c("Season", "TeamID_B")) %>% 
  mutate(
    diff_seed = Seed_A - Seed_B,
    diff_rating = Rating_A - Rating_B,
    diff_win_rate = win_rate_A - win_rate_B,
    diff_gap_avg = gap_avg_A - gap_avg_B,
    diff_score = Score_A-Score_B,
    target = ifelse(Score_A-Score_B>0, 1, 0)
  )

In [4]:
result_merged %>% head()

Season,DayNum,TeamID_A,Score_A,TeamID_B,Score_B,WLoc,NumOT,Seed_A,Rating_A,⋯,Seed_B,Rating_B,win_rate_B,gap_avg_B,diff_seed,diff_rating,diff_win_rate,diff_gap_avg,diff_score,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2010,134,1115,61,1457,44,N,0,16,44.92280,⋯,16,48.55172,0.5666667,0.6333333,0,-3.628926,-0.03541667,-0.9145833,17,1
2010,136,1124,68,1358,59,N,0,3,62.31469,⋯,14,53.03943,0.7500000,7.5714286,-11,9.275262,0.02419355,2.5898618,9,1
2010,136,1139,77,1431,59,N,0,5,62.29193,⋯,12,58.86788,0.8125000,11.5937500,-7,3.424057,0.06250000,-1.5312500,18,1
2010,136,1140,99,1196,92,N,2,7,60.70889,⋯,10,58.82181,0.6363636,6.1818182,-3,1.887081,0.21212121,10.7575758,7,1
2010,136,1242,90,1250,74,N,0,1,67.47625,⋯,16,48.69475,0.6875000,5.0937500,-15,18.781508,0.25367647,12.8768382,16,1
2010,136,1243,82,1317,62,N,0,2,62.29727,⋯,15,53.69969,0.7333333,3.5666667,-13,8.597580,0.04791667,6.6833333,20,1


In [5]:
result_merged %>% write_csv("train_mens.csv")